### Logistic Regression in Statsmodels

In [1]:
import pandas as pd
import statsmodels.formula.api as sm

In [2]:
# Load in the dataset
df = pd.read_csv("https://s3.amazonaws.com/demo-datasets/wine.csv")
df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,color,is_red,high_quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red,1.0,0.0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red,1.0,0.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red,1.0,0.0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red,1.0,0.0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red,1.0,0.0


#### Below, we fit a logistic regression model using statsmodels (patsy's) logistic regression formula

The formula says that the high_quality (coded as 1 or 0) DEPENDS on (~) following attributes:
 `residual_sugar, pH, and alcohol`.


In [4]:
# estimate parameters -- best fit line

model = sm.logit(
    "high_quality ~ residual_sugar + pH + alcohol",
    data = df
).fit()

model.summary()


Optimization terminated successfully.
         Current function value: 0.418431
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           high_quality   No. Observations:                 6497
Model:                          Logit   Df Residuals:                     6493
Method:                           MLE   Df Model:                            3
Date:                Mon, 27 Mar 2017   Pseudo R-squ.:                  0.1557
Time:                        15:27:41   Log-Likelihood:                -2718.5
converged:                       True   LL-Null:                       -3219.8
                                        LLR p-value:                5.039e-217
==================================================================================
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept        -11.7871      0.803    -14.674      0.000       -13.361   -10.213
residual_sugar     0.0471      0.009      5.441      0.000         0.030     0.064
pH                 0.1419      0.217      0.654      0.513        -0.283     0.567
alcohol            0.8946      0.031     28.600      0.000         0.833     0.956
==================================================================================
"""

In the table above
- `coef`, represents the coefficients we have learned for each feature
        For example, '0.8946' for alcohol represent the change in log odds 
        As alcohol content increases, the likelihood of high_quality increases
        
        We do not care about R^2 with these models--it is not relevant
        
        Table 1, last row of left side -- converged should always = True

In [6]:
import math
alcohol_coef = 0.8946

math.exp(alcohol_coef)
# As alcohol inreases by 1 unit, we expect that a wine 
# is 2.446 times as likely to be classified as 'high-quality'

## e ^ alcohol_coef is the ODDS RATIO
## alcohol_coef is the LOG-ODDS RATIO

2.4463570509072867

#### We can add interaction effects as well
- The `:` operator in patsy / formula-syntax represents when we care about two variables occurring together
- The `*` operator expands as follows: `a * b` expands to `a + b + a:b`, both of the original terms and interaction

In [7]:

model = sm.logit(
    "high_quality ~ residual_sugar * alcohol",
    data = df
).fit()

model.summary()


Optimization terminated successfully.
         Current function value: 0.416779
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           high_quality   No. Observations:                 6497
Model:                          Logit   Df Residuals:                     6493
Method:                           MLE   Df Model:                            3
Date:                Mon, 27 Mar 2017   Pseudo R-squ.:                  0.1590
Time:                        15:42:01   Log-Likelihood:                -2707.8
converged:                       True   LL-Null:                       -3219.8
                                        LLR p-value:                1.109e-221
==========================================================================================
                             coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                -13.0762      0.532    -24.600      0.000       -14.118   -12.034
residual_sugar             0.3601      0.066      5.431      0.000         0.230     0.490
alcohol                    1.0565      0.047     22.343      0.000         0.964     1.149
residual_sugar:alcohol    -0.0302      0.006     -4.763      0.000        -0.043    -0.018
==========================================================================================
"""